# pagaya-map_in_pandas

Easy python wrapper for Spark mapInPandas, applyInPandas

## Goal
Easily run legacy pandas-based python functions at scale on large amounts of data.

## applications
* Running legacy transformations, feature extraction etc.
* large scale model-evaluation
* large scale experiments and parameter tuning.
* A-B testing
* Concurrent training (e.g. xgboost)